In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e1/sample_submission.csv
/kaggle/input/playground-series-s5e1/train.csv
/kaggle/input/playground-series-s5e1/test.csv


In [5]:
# %load_ext cudf.pandas

In [6]:
!nvidia-smi

Thu Jan  2 19:36:59 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

In [8]:
pd.set_option('display.max_colwidth', None)

In [9]:
train= pd.read_csv('/kaggle/input/playground-series-s5e1/train.csv')
test= pd.read_csv('/kaggle/input/playground-series-s5e1/test.csv')
train = train.set_index('id')
test = test.set_index('id')
train = train.drop_duplicates()
test = test.drop_duplicates()
print("train_data shape :",train.shape)
print("test_data shape :",test.shape)

train_data shape : (230130, 5)
test_data shape : (98550, 4)


In [10]:
train.head(20)

,date,country,store,product,num_sold
id,,,,,
0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0
5,2010-01-01,Canada,Stickers for Less,Holographic Goose,300.0
6,2010-01-01,Canada,Stickers for Less,Kaggle,1837.0
7,2010-01-01,Canada,Stickers for Less,Kaggle Tiers,1659.0
8,2010-01-01,Canada,Stickers for Less,Kerneler,807.0


In [11]:
# train.describe().T

In [12]:
train.tail(20)

,date,country,store,product,num_sold
id,,,,,
230110,2016-12-31,Norway,Premium Sticker Mart,Holographic Goose,576.0
230111,2016-12-31,Norway,Premium Sticker Mart,Kaggle,3459.0
230112,2016-12-31,Norway,Premium Sticker Mart,Kaggle Tiers,2735.0
230113,2016-12-31,Norway,Premium Sticker Mart,Kerneler,1690.0
230114,2016-12-31,Norway,Premium Sticker Mart,Kerneler Dark Mode,1857.0
230115,2016-12-31,Singapore,Discount Stickers,Holographic Goose,200.0
230116,2016-12-31,Singapore,Discount Stickers,Kaggle,1259.0
230117,2016-12-31,Singapore,Discount Stickers,Kaggle Tiers,931.0
230118,2016-12-31,Singapore,Discount Stickers,Kerneler,556.0


In [13]:
train_info={
    'Data Type': train.dtypes,
    'Missing Values(MV)': train.isnull().sum(),
    'MV percentage(%)' : train.isnull().mean() * 100,
    'Unique Values': train.nunique()
}

train_summary = pd.DataFrame(train_info)
train_summary

,Data Type,Missing Values(MV),MV percentage(%),Unique Values
date,object,0,0.000000,2557
country,object,0,0.000000,6
store,object,0,0.000000,3
product,object,0,0.000000,5
num_sold,float64,8871,3.854778,4037


In [14]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

print(train.dtypes)
print('\n')
print(train.dtypes)

date        datetime64[ns]
country             object
store               object
product             object
num_sold           float64
dtype: object


date        datetime64[ns]
country             object
store               object
product             object
num_sold           float64
dtype: object


In [15]:
target = (set(train.columns) - set(test.columns)).pop()

print(f"Target column: {target}")
print(f"Data type: {train[target].dtype}")

Target column: num_sold
Data type: float64


In [16]:
print(train['country'].unique())
print(train['store'].unique())
print(train['product'].unique())

['Canada' 'Finland' 'Italy' 'Kenya' 'Norway' 'Singapore']
['Discount Stickers' 'Stickers for Less' 'Premium Sticker Mart']
['Holographic Goose' 'Kaggle' 'Kaggle Tiers' 'Kerneler'
 'Kerneler Dark Mode']


In [17]:
unique = {
    "Column": ['country', 'store', 'product'],
    "Value": ['Canada, Finland, Italy, Kenya, Norway, Singapore', 'Discount Stickers, Stickers for Less, Premium Sticker Mart', 'Holographic Goose, Kaggle, Kaggle Tiers, Kerneler, Kerneler Dark Mode'],
    "Count": [6, 3, 5]
}
unique = pd.DataFrame(unique)

print('The unique value of Original Train Dataset:\n')
unique

The unique value of Original Train Dataset:



,Column,Value,Count
0,country,"Canada, Finland, Italy, Kenya, Norway, Singapore",6
1,store,"Discount Stickers, Stickers for Less, Premium Sticker Mart",3
2,product,"Holographic Goose, Kaggle, Kaggle Tiers, Kerneler, Kerneler Dark Mode",5


### Feature Engineering

In [18]:
le = LabelEncoder()
object_cols = train.select_dtypes(include=['object']).columns
for col in object_cols:
    train[col] = le.fit_transform(train[col])    
    test[col] = le.transform(test[col])

print(train.dtypes.T)
print('\n')
print(test.dtypes.T)

date        datetime64[ns]
country              int64
store                int64
product              int64
num_sold           float64
dtype: object


date       datetime64[ns]
country             int64
store               int64
product             int64
dtype: object


In [19]:
def create_features(data):
    """
    Create time series features based on time series index.
    """
    data = data.copy()
    data['year'] = data['date'].dt.year
    data['month'] = data['date'].dt.month
    data['day'] = data['date'].dt.day
    data['weekday'] = data['date'].dt.weekday  # 0 = Monday, 6 = Sunday
    data['quarter'] = data['date'].dt.quarter

    # Interaction Features: Combine categorical variables
    data['country_store'] = data['country'] * 10 + data['store']
    data['store_product'] = data['store'] * 10 + data['product']
    data['country_product'] = data['country'] * 10 + data['product']
    
    # Cyclic Features: Encode seasonality for 'month' and 'weekday'
    data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
    data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)
    data['weekday_sin'] = np.sin(2 * np.pi * data['weekday'] / 7)
    data['weekday_cos'] = np.cos(2 * np.pi * data['weekday'] / 7)
    
    return data

train = create_features(train)
train.head(10)

,date,country,store,product,num_sold,year,month,day,weekday,quarter,country_store,store_product,country_product,month_sin,month_cos,weekday_sin,weekday_cos
id,,,,,,,,,,,,,,,,,
0,2010-01-01,0,0,0,NaN,2010,1,1,4,1,0,0,0,0.5,0.866025,-0.433884,-0.900969
1,2010-01-01,0,0,1,973.0,2010,1,1,4,1,0,1,1,0.5,0.866025,-0.433884,-0.900969
2,2010-01-01,0,0,2,906.0,2010,1,1,4,1,0,2,2,0.5,0.866025,-0.433884,-0.900969
3,2010-01-01,0,0,3,423.0,2010,1,1,4,1,0,3,3,0.5,0.866025,-0.433884,-0.900969
4,2010-01-01,0,0,4,491.0,2010,1,1,4,1,0,4,4,0.5,0.866025,-0.433884,-0.900969
5,2010-01-01,0,2,0,300.0,2010,1,1,4,1,2,20,0,0.5,0.866025,-0.433884,-0.900969
6,2010-01-01,0,2,1,1837.0,2010,1,1,4,1,2,21,1,0.5,0.866025,-0.433884,-0.900969
7,2010-01-01,0,2,2,1659.0,2010,1,1,4,1,2,22,2,0.5,0.866025,-0.433884,-0.900969
8,2010-01-01,0,2,3,807.0,2010,1,1,4,1,2,23,3,0.5,0.866025,-0.433884,-0.900969


In [20]:
train.tail(10)

,date,country,store,product,num_sold,year,month,day,weekday,quarter,country_store,store_product,country_product,month_sin,month_cos,weekday_sin,weekday_cos
id,,,,,,,,,,,,,,,,,
230120,2016-12-31,5,2,0,384.0,2016,12,31,5,4,52,20,50,-2.449294e-16,1.0,-0.974928,-0.222521
230121,2016-12-31,5,2,1,2380.0,2016,12,31,5,4,52,21,51,-2.449294e-16,1.0,-0.974928,-0.222521
230122,2016-12-31,5,2,2,2103.0,2016,12,31,5,4,52,22,52,-2.449294e-16,1.0,-0.974928,-0.222521
230123,2016-12-31,5,2,3,1052.0,2016,12,31,5,4,52,23,53,-2.449294e-16,1.0,-0.974928,-0.222521
230124,2016-12-31,5,2,4,1312.0,2016,12,31,5,4,52,24,54,-2.449294e-16,1.0,-0.974928,-0.222521
230125,2016-12-31,5,1,0,466.0,2016,12,31,5,4,51,10,50,-2.449294e-16,1.0,-0.974928,-0.222521
230126,2016-12-31,5,1,1,2907.0,2016,12,31,5,4,51,11,51,-2.449294e-16,1.0,-0.974928,-0.222521
230127,2016-12-31,5,1,2,2299.0,2016,12,31,5,4,51,12,52,-2.449294e-16,1.0,-0.974928,-0.222521
230128,2016-12-31,5,1,3,1242.0,2016,12,31,5,4,51,13,53,-2.449294e-16,1.0,-0.974928,-0.222521


In [21]:
test = create_features(test)
test.head(10)

,date,country,store,product,year,month,day,weekday,quarter,country_store,store_product,country_product,month_sin,month_cos,weekday_sin,weekday_cos
id,,,,,,,,,,,,,,,,
230130,2017-01-01,0,0,0,2017,1,1,6,1,0,0,0,0.5,0.866025,-0.781831,0.62349
230131,2017-01-01,0,0,1,2017,1,1,6,1,0,1,1,0.5,0.866025,-0.781831,0.62349
230132,2017-01-01,0,0,2,2017,1,1,6,1,0,2,2,0.5,0.866025,-0.781831,0.62349
230133,2017-01-01,0,0,3,2017,1,1,6,1,0,3,3,0.5,0.866025,-0.781831,0.62349
230134,2017-01-01,0,0,4,2017,1,1,6,1,0,4,4,0.5,0.866025,-0.781831,0.62349
230135,2017-01-01,0,2,0,2017,1,1,6,1,2,20,0,0.5,0.866025,-0.781831,0.62349
230136,2017-01-01,0,2,1,2017,1,1,6,1,2,21,1,0.5,0.866025,-0.781831,0.62349
230137,2017-01-01,0,2,2,2017,1,1,6,1,2,22,2,0.5,0.866025,-0.781831,0.62349
230138,2017-01-01,0,2,3,2017,1,1,6,1,2,23,3,0.5,0.866025,-0.781831,0.62349


In [22]:
print(train.nunique())
print('\n')
print(test.nunique())

date               2557
country               6
store                 3
product               5
num_sold           4037
year                  7
month                12
day                  31
weekday               7
quarter               4
country_store        18
store_product        15
country_product      30
month_sin            11
month_cos            11
weekday_sin           7
weekday_cos           7
dtype: int64


date               1095
country               6
store                 3
product               5
year                  3
month                12
day                  31
weekday               7
quarter               4
country_store        18
store_product        15
country_product      30
month_sin            11
month_cos            11
weekday_sin           7
weekday_cos           7
dtype: int64


In [23]:
correlation_matrix = train.corr()

# Display a summary of the new features and correlation matrix
print(correlation_matrix['num_sold'].sort_values(ascending=False))

num_sold           1.000000
store              0.239883
store_product      0.237983
country_store      0.140422
country_product    0.129382
country            0.129113
weekday            0.069613
month_sin          0.014119
weekday_cos        0.013453
product            0.004255
day                0.001137
month_cos         -0.001781
month             -0.006255
quarter           -0.006670
year              -0.040462
date              -0.040936
weekday_sin       -0.063575
Name: num_sold, dtype: float64


In [24]:
# scaler = StandardScaler()
# train['num_sold_scaled'] = scaler.fit_transform(train[['num_sold']])
# train

### Model Building

In [25]:
X = train.drop(['num_sold'], axis=1)
y = train['num_sold']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Create a scorer for GridSearchCV
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [27]:
# Define hyperparameter grid
param_grid = {
    'n_estimators': [100, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2]
}

xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

In [28]:
# Perform Grid Search with Cross-Validation
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring=mape_scorer,
    cv=5,
    verbose=1,
    n_jobs=-1
)

# Fit grid search
grid_search.fit(X_train, y_train)

# Best parameters and score
best_params = grid_search.best_params_
best_score = abs(grid_search.best_score_)

Fitting 5 folds for each of 6561 candidates, totalling 32805 fits


ValueError: 
All the 32805 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
32805 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 1055, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 521, in _wrap_evaluation_matrices
    train_dmatrix = create_dmatrix(
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 95

## Forecast on Test